In [1]:
from py_lex import Liwc, EmoLex
import nltk
import pandas as pd
# nltk.download('punkt')

In [2]:
data = pd.read_json('./data/merged_data_SA.json')

In [3]:
data.sample()

,index,total_awards_received,approved_at_utc,ups,awarders,mod_reason_by,banned_by,author_flair_type,removal_reason,link_id,...,subreddit_name_prefixed,controversiality,depth,author_flair_background_color,mod_reports,mod_note,distinguished,author_cakeday,sent_polarity,sent_subjectivity
8899,265,0,NaN,6,[],NaN,NaN,text,NaN,t3_died0t,...,r/Beto2020,0,0,,[],NaN,None,NaN,0.0,0.0


In [4]:
subreddits = data['subreddit'].unique()
subreddits

array(['AmericanGovernment', 'AmericanPolitics', 'bernie',
       'BernieSanders', 'Beto2020', 'BetoORourke', 'bidenbro', 'BillWeld',
       'business', 'Communist', 'Conservative', 'conservatives',
       'corybooker', 'Delaney2020', 'democracy', 'democrats',
       'Democrats2020', 'Economics', 'education', 'egalitarian',
       'ElizabethWarren', 'energy', 'environment', 'government',
       'history', 'inslee2020', 'JoeBiden', 'JoeWalsh', 'Kamala',
       'Kossacks_for_Sanders', 'law', 'LGBTnews', 'Liberal',
       'Libertarian', 'LibertarianLeft', 'LibertarianSocialism'],
      dtype=object)

# Test

In [7]:
lexicon = EmoLex('./NRC-Sentiment-Emotion-Lexicons/NRC-Sentiment-Emotion-Lexicons/NRC-Emotion-Lexicon-v0.92/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt')
stopset = set(nltk.corpus.stopwords.words('English'))

In [8]:
body = data.sample()['body'].values[0]
body

'How can the marginal cost of production hit zero? I have not read Marx.'

In [9]:
doc = nltk.tokenize.word_tokenize(body)
doc = [word.lower() for word in doc if not word in stopset]
lexicon.summarize_doc(doc)

{'fear': 0.0,
 'disgust': 0.0,
 'trust': 0.0,
 'surprise': 0.0,
 'joy': 0.0,
 'sadness': 0.0,
 'anticipation': 0.1111111111111111,
 'positive': 0.1111111111111111,
 'anger': 0.1111111111111111,
 'negative': 0.1111111111111111}

In [200]:
for index, row in data.iterrows():
    print(row.body)
    break

Is murder illegal?

That's the point.


# Implement on the whole dataset

In [21]:
def get_scores(row):
    return lexicon.summarize_doc([word.lower() for word in nltk.tokenize.word_tokenize(row['body']) if not word in stopset])

In [29]:
data['scores'] = data.apply(get_scores, axis=1)

In [37]:
for score_name in ['anticipation', 'positive', 'trust', 'joy', 'fear', 'negative', 'sadness', 'disgust', 'anger', 'surprise']:
    data['sc_'+score_name] = data['scores'].apply(lambda x: x[score_name])

In [39]:
data.head()

,index,total_awards_received,approved_at_utc,ups,awarders,mod_reason_by,banned_by,author_flair_type,removal_reason,link_id,...,sc_anticipation,sc_positive,sc_trust,sc_joy,sc_fear,sc_negative,sc_sadness,sc_disgust,sc_anger,sc_surprise
0,0,0,NaN,0,[],NaN,NaN,text,NaN,t3_d88slw,...,0.000000,0.000000,0.0,0.000000,0.400000,0.400000,0.400000,0.400000,0.400000,0.200000
1,1,0,NaN,1,[],NaN,NaN,text,NaN,t3_d88slw,...,0.041667,0.041667,0.0,0.000000,0.125000,0.125000,0.083333,0.041667,0.125000,0.041667
2,2,0,NaN,1,[],NaN,NaN,text,NaN,t3_d88slw,...,0.034483,0.034483,0.0,0.034483,0.137931,0.172414,0.137931,0.103448,0.103448,0.034483
3,3,0,NaN,1,[],NaN,NaN,text,NaN,t3_d88slw,...,0.100000,0.200000,0.2,0.000000,0.300000,0.100000,0.100000,0.100000,0.100000,0.100000
4,4,0,NaN,1,[],NaN,NaN,text,NaN,t3_d88slw,...,0.000000,0.000000,0.0,0.000000,0.375000,0.500000,0.500000,0.375000,0.375000,0.250000


In [43]:
data.to_json('./data/merged_data_SA.json')